In [1]:
!rm -rf dqm_repo
!feast init dqm_repo
%cd dqm_repo


Creating a new Feast repository in /Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo.

/Users/franciscojavierarceo/GitHub/feast/examples/data-quality-monitoring/dqm_repo


In [2]:
import pandas as pd

pd.read_parquet("data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-07-26 21:00:00+00:00,1005,0.675813,0.697865,463,2022-08-10 21:06:22.761
1,2022-07-26 22:00:00+00:00,1005,0.707445,0.642702,927,2022-08-10 21:06:22.761
2,2022-07-26 23:00:00+00:00,1005,0.237603,0.195165,366,2022-08-10 21:06:22.761
3,2022-07-27 00:00:00+00:00,1005,0.111497,0.292317,410,2022-08-10 21:06:22.761
4,2022-07-27 01:00:00+00:00,1005,0.965894,0.930423,816,2022-08-10 21:06:22.761
...,...,...,...,...,...,...
1802,2022-08-10 19:00:00+00:00,1001,0.229680,0.490494,993,2022-08-10 21:06:22.761
1803,2022-08-10 20:00:00+00:00,1001,0.025623,0.181108,430,2022-08-10 21:06:22.761
1804,2021-04-12 07:00:00+00:00,1001,0.201743,0.193218,517,2022-08-10 21:06:22.761
1805,2022-08-03 09:00:00+00:00,1003,0.072511,0.965429,690,2022-08-10 21:06:22.761


In [3]:
!feast apply

Created entity driver
Created feature view driver_hourly_stats
Created feature service driver_activity

Created sqlite table dqm_repo_driver_hourly_stats



In [4]:
from datetime import datetime, timedelta
import pandas as pd

from feast import FeatureStore
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage
from feast.dqm.profilers.ge_profiler import ge_profiler
from great_expectations.dataset import PandasDataset
from great_expectations.core.expectation_suite import ExpectationSuite

# The entity dataframe is the dataframe we want to enrich with feature values
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "label_driver_reported_satisfaction": [1, 5, 3], 
        "event_timestamp": [
            datetime.now() - timedelta(minutes=11),
            datetime.now() - timedelta(minutes=36),
            datetime.now() - timedelta(minutes=73),
        ],
    }
)

store = FeatureStore(repo_path=".")

training_data_job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
)

In [5]:
training_data_job.to_df()

,driver_id,label_driver_reported_satisfaction,event_timestamp,conv_rate,acc_rate,avg_daily_trips
0,1003,3,2022-08-10 19:53:51.637185+00:00,0.754099,0.830307,823
1,1002,5,2022-08-10 20:30:51.637183+00:00,0.025148,0.506514,164
2,1001,1,2022-08-10 20:55:51.637174+00:00,0.025623,0.181108,430


In [6]:
reference_dataset = store.create_saved_dataset(
    from_=training_data_job,
    name="reference_dataset",
    storage=SavedDatasetFileStorage(path='data/driver_stats.parquet')
)

reference_dataset

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:1127: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


<SavedDataset(name = reference_dataset, features = ['driver_hourly_stats:conv_rate', 'driver_hourly_stats:acc_rate', 'driver_hourly_stats:avg_daily_trips'], join_keys = ['driver_id', 'label_driver_reported_satisfaction'], storage = <feast.infra.offline_stores.file_source.SavedDatasetFileStorage object at 0x13f124bb0>, full_feature_names = False, tags = {}, feature_service_name = None, _retrieval_job = <feast.infra.offline_stores.file.FileRetrievalJob object at 0x13f1248e0>, min_event_timestamp = 2022-08-10 19:53:51.637185+00:00, max_event_timestamp = 2022-08-10 20:55:51.637174+00:00, created_timestamp = 2022-08-11 03:06:51.854923, last_updated_timestamp = 2022-08-11 03:06:51.854923)>

In [7]:
@ge_profiler
def user_features_profiler(dataset: PandasDataset) -> ExpectationSuite:
    print(dataset.columns)
    dataset.expect_column_to_exist("driver_id")
    dataset.expect_column_values_to_be_between("driver_hourly_stats__avg_daily_trips", 0, 1000)
    dataset.expect_column_values_to_be_between("driver_hourly_stats__conv_rate", 0, 1)
    dataset.expect_column_values_to_be_between("driver_hourly_stats__acc_rate", 0, 1)
    return dataset.get_expectation_suite()

ds = store.get_saved_dataset('reference_dataset')

/Users/franciscojavierarceo/GitHub/feast/sdk/python/feast/feature_store.py:1180: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [8]:
validation_reference = ds.as_reference(name='user_features_profiler', profiler=user_features_profiler)
_ = training_data_job.to_df(validation_reference=validation_reference)

KeyError: "['created'] not in index"

In [ ]:
training_data_job.to_df()